<h1 style="color:red">Test data</h1>

In [9]:
import pandas as pd
# Load training and testing data
test = pd.read_csv("house_prices_data/test.csv")

In [10]:
# Display the first few rows of the training data
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


<p style="color:red">Notice how the train data had 81 columns and test data has 80 columns.<br>
This is because our target variable <b>'Sale Price'</b>, is to be used in training.</p>

In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [12]:
# Calculate the number and percentage of missing values
missing_values = test.isnull().sum()
missing_percentage = (missing_values / len(test)) * 100

# Create a DataFrame to summarize missing values
missing_summary = pd.DataFrame({
    'Column': test.columns,
    'Missing_Count': missing_values,
    'Missing_Percentage': missing_percentage
}).sort_values(by='Missing_Percentage', ascending=False)

# Display columns with missing values
missing_summary = missing_summary[missing_summary['Missing_Count'] > 0]
missing_summary.reset_index(drop=True, inplace=True)
print(missing_summary)

          Column  Missing_Count  Missing_Percentage
0         PoolQC           1456           99.794380
1    MiscFeature           1408           96.504455
2          Alley           1352           92.666210
3          Fence           1169           80.123372
4     MasVnrType            894           61.274846
5    FireplaceQu            730           50.034270
6    LotFrontage            227           15.558602
7    GarageYrBlt             78            5.346127
8     GarageQual             78            5.346127
9   GarageFinish             78            5.346127
10    GarageCond             78            5.346127
11    GarageType             76            5.209047
12      BsmtCond             45            3.084304
13      BsmtQual             44            3.015764
14  BsmtExposure             44            3.015764
15  BsmtFinType1             42            2.878684
16  BsmtFinType2             42            2.878684
17    MasVnrArea             15            1.028101
18      MSZo

<h3 style="color:lime;">Test dataset shows a very similar pattern of missing values to the training dataset. Here's an analysis based on these results:</h3>

### Columns to drop:
1. **`PoolQC` (99.79%)**: Too much missing data to be useful.
2. **`MiscFeature` (96.50%)**: Similar to the training set, it has limited data and relevance.
3. **`Alley` (92.67%)**: High percentage of missing values.
4. **`Fence` (80.12%)**: Over 80% missing data makes it impractical for analysis.

### Columns to consider filling:
1. **`MasVnrType` (61.27%)**: Can be treated similarly to the training set—impute with mode or use "None" to indicate missing masonry veneer type.
2. **`FireplaceQu` (50.03%)**: Likely indicates no fireplace, so fill with "None."
3. **`LotFrontage` (15.56%)**: Impute using the median or by neighborhood.
4. **Garage-related columns** (`GarageYrBlt`, `GarageQual`, `GarageFinish`, `GarageCond`, `GarageType`): Missing values (~5.35%) likely represent "No Garage" and can be imputed with appropriate placeholders.
5. **Basement-related columns** (`BsmtCond`, `BsmtQual`, `BsmtExposure`, `BsmtFinType1`, `BsmtFinType2`): Missing values (~3%) likely represent "No Basement" and can be imputed with placeholders.
6. **`MasVnrArea` (1.03%)**: Can be imputed with 0 or the median value.
7. **`MSZoning` (0.27%)**: Fill with mode as it's categorical.
8. **Other very low missing columns** (e.g., `Utilities`, `Functional`, `KitchenQual`): Impute with mode or specific logic relevant to the column.

### Additional Columns with Single Missing Values:
These columns each have 1 missing value (0.068%), so they can be filled based on logical imputations:
- **`BsmtFinSF1`, `BsmtFinSF2`, `BsmtUnfSF`, `TotalBsmtSF`**: Likely indicate no basement or zero square footage.
- **`Exterior1st`, `Exterior2nd`**: Can be filled with mode.
- **`GarageCars`, `GarageArea`**: Likely represent no garage, so fill with 0.
- **`SaleType`**: Fill with mode.



<h2 style="color:red;">CLEANING</h2>

In [13]:
# List of columns to drop for the test dataset
drop_columns_test = ['PoolQC', 'MiscFeature', 'Alley', 'Fence','Id'] #Dont need Id rn it is for unique identification only,so dropping it

# Dropping the columns from the test dataset
test = test.drop(columns=drop_columns_test)

# Check the result
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   int64  
 1   MSZoning       1455 non-null   object 
 2   LotFrontage    1232 non-null   float64
 3   LotArea        1459 non-null   int64  
 4   Street         1459 non-null   object 
 5   LotShape       1459 non-null   object 
 6   LandContour    1459 non-null   object 
 7   Utilities      1457 non-null   object 
 8   LotConfig      1459 non-null   object 
 9   LandSlope      1459 non-null   object 
 10  Neighborhood   1459 non-null   object 
 11  Condition1     1459 non-null   object 
 12  Condition2     1459 non-null   object 
 13  BldgType       1459 non-null   object 
 14  HouseStyle     1459 non-null   object 
 15  OverallQual    1459 non-null   int64  
 16  OverallCond    1459 non-null   int64  
 17  YearBuilt      1459 non-null   int64  
 18  YearRemo

In [14]:
test.shape

(1459, 75)

<h3>1. Identifying numerical and categorical columns with missing values:</h3>

In [15]:
# Identify numerical columns with missing values
numerical_columns = test.select_dtypes(include=['float64', 'int64']).columns
numerical_missing_test = test[numerical_columns].isnull().sum()

# Identify categorical columns with missing values
categorical_columns = test.select_dtypes(include=['object']).columns
categorical_missing_test = test[categorical_columns].isnull().sum()

In [16]:
# Print the columns with missing values
numerical_missing_test[numerical_missing_test > 0]

LotFrontage     227
MasVnrArea       15
BsmtFinSF1        1
BsmtFinSF2        1
BsmtUnfSF         1
TotalBsmtSF       1
BsmtFullBath      2
BsmtHalfBath      2
GarageYrBlt      78
GarageCars        1
GarageArea        1
dtype: int64

In [17]:
# Print the columns with missing values
categorical_missing_test[categorical_missing_test > 0]

MSZoning          4
Utilities         2
Exterior1st       1
Exterior2nd       1
MasVnrType      894
BsmtQual         44
BsmtCond         45
BsmtExposure     44
BsmtFinType1     42
BsmtFinType2     42
KitchenQual       1
Functional        2
FireplaceQu     730
GarageType       76
GarageFinish     78
GarageQual       78
GarageCond       78
SaleType          1
dtype: int64